## Building Machine Learning Algorithem to  Predict Churn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loading the dataset to a Pandas DataFrame
import pandas as pd
import numpy as np
# Read the Excel file into a Pandas DataFrame
# Make sure the path to the file is correct. If the file is in the same directory as the notebook, you can use just the file name.
# If it's in a different directory, provide the full path.
df = pd.read_csv('/content/drive/MyDrive/Group_Project_ChurnData/ChurnData.csv')
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [ ]:
# conver t gender varibles as 0,1 outcomes

df['gender'].replace(['Female', 'Male'], [0, 1], inplace=True)


In [ ]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,1,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [ ]:
x = df.drop('Churn', axis=1)
y = df['Churn']
print(y.value_counts())

Churn
No     5174
Yes    1869
Name: count, dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   int64  
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:


# Ensure all 'gender' values are converted to strings before replacement
df['gender'] = df['gender'].astype(str)
df['gender'].replace(['Female', 'Male'], [0, 1], inplace=True)

# Verify the conversion
print(df['gender'].unique())

['0' '1']


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [ ]:

print ("Number of rows in the dataset  : " ,df.shape[0])
print ("Number of Columns in the dataset : " ,df.shape[1])
print ("Number of Features : \n" ,df.columns.tolist())
print ("Missing values :  ", df.isnull().sum().values.sum())
print ("Unique values :  \n",df.nunique())

Number of rows in the dataset  :  7043
Number of Columns in the dataset :  21
Number of Features : 
 ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
Missing values :   0
Unique values :  
 customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int

In [ ]:
df['TotalCharges']=df["TotalCharges"].replace(r'\s+',np.nan,regex=True)
df['TotalCharges']=pd.to_numeric(df['TotalCharges'])

In [ ]:
fill=df.MonthlyCharges*df.tenure

In [ ]:
df.TotalCharges.fillna(fill,inplace=True)

Now the changing the datatype of the columns from object to numerical datat type. And also doing the one hot encoding and label encoding to the columns of the data set which contains the categorical data.

In [ ]:
def changeColumnsToString(df):
    columnsNames=['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
    for col in columnsNames:
        df[col]=df[col].astype('str').str.replace('Yes','1').replace('No','0').replace('No internet service','0').replace('No phone service',0)

changeColumnsToString(df)

df['SeniorCitizen']=df['SeniorCitizen'].astype(bool)
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')

In [ ]:
print("Payment methods: ",df.PaymentMethod.unique())
print("Contract types: ",df.Contract.unique())
print("Gender: ",df.gender.unique())
print("Senior Citizen: ",df.SeniorCitizen.unique())
print("Internet Service Types: ",df.InternetService.unique())

Payment methods:  ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']
Categories (4, object): ['Bank transfer (automatic)', 'Credit card (automatic)', 'Electronic check',
                         'Mailed check']
Contract types:  ['Month-to-month', 'One year', 'Two year']
Categories (3, object): ['Month-to-month', 'One year', 'Two year']
Gender:  ['0', '1']
Categories (2, object): ['0', '1']
Senior Citizen:  [False  True]
Internet Service Types:  ['DSL', 'Fiber optic', 'No']
Categories (3, object): ['DSL', 'Fiber optic', 'No']


In [ ]:
df['gender']=df['gender'].astype('category')
df['PaymentMethod']=df['PaymentMethod'].astype('category')
df['Contract']=df['Contract'].astype('category')
df['SeniorCitizen']=df['SeniorCitizen'].astype('category')
df['InternetService']=df['InternetService'].astype('category')
#df.dtypes

In [ ]:
df['Churn'].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

## Note that this data set is highly imbalance so we are going to use SMOTE technique for ML model bulding and further more analysis

In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd

# Assuming 'x' is your DataFrame containing features
x_encoded = pd.get_dummies(x)  # One-hot encode categorical features

smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x_encoded, y)

In [ ]:
X=x_smote
y=y_smote


In [ ]:
## Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(x_smote.shape,X_train.shape,X_test.shape)

(10348, 13618) (8278, 13618) (2070, 13618)


In [ ]:
## Model Training
logistic_model = LogisticRegression(max_iter=1000)
# training the LogisticRegression model with Training data
logistic_model .fit(X_train, y_train)
# Make predictions with the Logistic Regression model
y_pred_logistic = logistic_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
## Confution matrix for logistic regression model
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_logistic)
print(cm)



[[908 113]
 [146 903]]


In [ ]:
## Performance matric for the logistic regression model
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_logistic))


              precision    recall  f1-score   support

           0       0.86      0.89      0.88      1021
           1       0.89      0.86      0.87      1049

    accuracy                           0.87      2070
   macro avg       0.88      0.88      0.87      2070
weighted avg       0.88      0.87      0.87      2070



In [ ]:
## Logistic regression model evaluation
# accuracy on training data
X_train_prediction = logistic_model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print('Accuracy on training data for LogisticRegression: ', training_data_accuracy)

Accuracy on training data for LogisticRegression:  0.9568736409760812


In [ ]:
## Logistic regression testing data evaluation
X_test_prediction = logistic_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print('Accuracy on testing data for LogisticRegression: ', test_data_accuracy)

Accuracy on testing data for LogisticRegression:  0.8748792270531401


In [ ]:
## Using random forest to model training
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [ ]:
## Confution matrix for random forest
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[899 122]
 [131 918]]


In [ ]:
## Performance matric for Random forest
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.88      0.88      1021
           1       0.88      0.88      0.88      1049

    accuracy                           0.88      2070
   macro avg       0.88      0.88      0.88      2070
weighted avg       0.88      0.88      0.88      2070



In [ ]:
# Calculate Specificity for Random Forest
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

Specificity: 0.8805093046033301


In [ ]:
## Random forest model evaluation
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print('Accuracy on training data for Random Forest : ', training_data_accuracy)

Accuracy on training data for Random Forest :  1.0


In [ ]:
## Random forest testing data evaluation
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print('Accuracy on testing data for Random Forest : ', test_data_accuracy)

Accuracy on testing data for Random Forest :  0.8777777777777778


In [ ]:
## Using KNN for model training
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


KNeighborsClassifier()

In [ ]:
## KNN model training data evaluation
X_train_prediction = knn.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print('Accuracy on training data for KNN : ', training_data_accuracy)



Accuracy on training data for KNN :  0.9208746073930901


In [ ]:
## KNN model testing data evaluation
X_test_prediction = knn.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print('Accuracy on testing data for KNN : ', test_data_accuracy)

Accuracy on testing data for KNN :  0.8690821256038648


# Conclusion
* Good Performance: Logistic Regression achieves a high accuracy on the test data (87.49%), very close to the best-performing Random Forest model (87.78%).
* The gap between its training accuracy (95.69%) and testing accuracy (87.49%) is smaller compared to Random Forest and KNN.
# As conclusion, The Logistic Regression model, with its combination of good performance, generalization, and interpretability, appears to be the most suitable choice for this telecom customer churn prediction task.